In [1]:
!pip install --upgrade elasticsearch

     |████████████████████████████████| 92kB 2.4MB/s 


In [2]:
!pip install dateparser

     |████████████████████████████████| 358kB 2.8MB/s 


In [0]:
import os, base64, re

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from elasticsearch import Elasticsearch
from elasticsearch import helpers

# to process funky dates
from datetime import datetime, timedelta
import dateparser

# to geocode location
import requests
import json

# feature transformation pipelines and modelling
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler, PowerTransformer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.svm import SVC #LinearSVC, NuSVC
from sklearn.decomposition import TruncatedSVD
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction import DictVectorizer

from sklearn.model_selection import train_test_split


stopwords = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]

# elasticsearch credentials
bonsai_url = ''
bonsai_port = 443

# google maps credentials
GOOGLE_MAPS_API_URL = 'https://maps.googleapis.com/maps/api/geocode/json'
api_key = ""

In [32]:
!wget -P /ieee8023/  https://raw.githubusercontent.com/ieee8023/covid-chestxray-dataset/master/metadata.csv

--2020-04-18 13:19:21--  https://raw.githubusercontent.com/ieee8023/covid-chestxray-dataset/master/metadata.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 192250 (188K) [text/plain]
Saving to: ‘/ieee8023/metadata.csv.1’

metadata.csv.1      100%[===================>] 187.74K  --.-KB/s    in 0.04s   

2020-04-18 13:19:21 (4.20 MB/s) - ‘/ieee8023/metadata.csv.1’ saved [192250/192250]



In [33]:
df_demog = pd.read_csv('/ieee8023/metadata.csv', encoding='utf8')
df_demog.head()

,patientid,offset,sex,age,finding,survival,intubated,intubation_present,went_icu,needed_supplemental_O2,extubated,temperature,pO2_saturation,leukocyte_count,neutrophil_count,lymphocyte_count,view,modality,date,location,folder,filename,doi,url,license,clinical_notes,other_notes,Unnamed: 27
0,2,0.0,M,65.0,COVID-19,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PA,X-ray,"January 22, 2020","Cho Ray Hospital, Ho Chi Minh City, Vietnam",images,auntminnie-a-2020_01_28_23_51_6665_2020_01_28_...,10.1056/nejmc2001272,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,infiltrate in the upper lobe of the left lung,NaN,NaN
1,2,3.0,M,65.0,COVID-19,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PA,X-ray,"January 25, 2020","Cho Ray Hospital, Ho Chi Minh City, Vietnam",images,auntminnie-b-2020_01_28_23_51_6665_2020_01_28_...,10.1056/nejmc2001272,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,progressive infiltrate and consolidation,NaN,NaN
2,2,5.0,M,65.0,COVID-19,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PA,X-ray,"January 27, 2020","Cho Ray Hospital, Ho Chi Minh City, Vietnam",images,auntminnie-c-2020_01_28_23_51_6665_2020_01_28_...,10.1056/nejmc2001272,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,progressive infiltrate and consolidation,NaN,NaN
3,2,6.0,M,65.0,COVID-19,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PA,X-ray,"January 28, 2020","Cho Ray Hospital, Ho Chi Minh City, Vietnam",images,auntminnie-d-2020_01_28_23_51_6665_2020_01_28_...,10.1056/nejmc2001272,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,progressive infiltrate and consolidation,NaN,NaN
4,4,0.0,F,52.0,COVID-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PA,X-ray,"January 25, 2020","Changhua Christian Hospital, Changhua City, Ta...",images,nejmc2001573_f1a.jpeg,10.1056/NEJMc2001573,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,diffuse infiltrates in the bilateral lower lungs,NaN,NaN


In [0]:
df_demog['clinical_notes'] = df_demog['clinical_notes'].fillna("")
df_demog['offset'] = df_demog['offset'].fillna(0).apply(lambda x: int(x))
df_demog['age'] = df_demog['age'].fillna(0).apply(lambda x: int(x))
df_demog['sex'] = df_demog['sex'].fillna('unknown')
df_demog['survival'] = df_demog['survival'].fillna('Unknown')
df_demog['intubated'] = df_demog['intubated'].fillna('Unknown')
df_demog['pO2_saturation'] = df_demog['pO2_saturation'].fillna(0.0)
df_demog['temperature'] = df_demog['temperature'].fillna(36.5).apply(lambda x: (x-32)*5/9 if x>=90 else x)
df_demog['location'] = df_demog['location'].fillna("")

In [0]:
df_demog=df_demog.drop_duplicates()

In [19]:
#create a list of manually imputted locations - to create a lookup table
locations = df_demog[df_demog['location'] != ''][
    ['location','patientid']
    ].groupby(['location']).count().reset_index()['location'].values

locations

array(['Australia', 'Barcelona, Spain ', 'Cairo, Egypt',
       'Canterbury, United Kingdom ',
       'Changhua Christian Hospital, Changhua City, Taiwan ', 'China',
       'Cho Ray Hospital, Ho Chi Minh City, Vietnam',
       'Civili Hospital, Vigevano, Italy',
       'GZA Sint-Augustinus, Antwerp, Belgium',
       'Hospital Universitario Doctor Peset, Valencia, Spain',
       'Hospital Universitario, Madrid, Spain',
       'Hospital of Wuhan University, Wuhan, China', 'Iran', 'Italy',
       "Jiangxi Provincial People's Hospital, Nanchang, China",
       'Jinling Hospital, Medical School of Nanjing University, Nanjing, Jiangsu, China',
       'Jönköping, Sweden', 'Laniado Hospital, Netanya, Israel',
       'Macao, China', 'Melbourne, Australia',
       'Mount Sinai Hospital, Toronto, Ontario, Canada',
       'Myongji Hospital, Goyang, Korea', 'Nottingham, United Kingdom ',
       'Orange, California, USA', 'Ospedale Santo Spirito. Rome, Italy',
       'Ospedale di Cattinara, Trieste,

In [0]:
{'results': [
    {'address_components': [
                            {'long_name': 'Canterbury',
                            'short_name': 'Canterbury',
                            'types': ['locality', 'political']
                             },
              {'long_name': 'Canterbury',
              'short_name': 'Canterbury',
              'types': ['postal_town']
               },
    {'long_name': 'Kent',
     'short_name': 'Kent',
     'types': ['administrative_area_level_2', 'political']},

    {'long_name': 'England',
     'short_name': 'England',
     'types': ['administrative_area_level_1', 'political']},

    {'long_name': 'United Kingdom',
     'short_name': 'GB',
     'types': ['country', 'political']}
     ],

   'formatted_address': 'Canterbury, UK',
   'geometry': {'bounds': {'northeast': {'lat': 51.3061023, 'lng': 1.1237432},
     'southwest': {'lat': 51.25098550000001, 'lng': 1.0439585}},
    'location': {'lat': 51.280233, 'lng': 1.0789089},
    'location_type': 'APPROXIMATE',
    'viewport': {'northeast': {'lat': 51.3061023, 'lng': 1.1237432},
     'southwest': {'lat': 51.25098550000001, 'lng': 1.0439585}}},
   'place_id': 'ChIJiSyDDcVK2UcRivmDN1Pl8Z4',
   'types': ['locality', 'political']}],
 'status': 'OK'}

In [20]:
loc = locations[4]

params = {
    'address': loc,
    'key' : api_key    
    }
req = requests.get(GOOGLE_MAPS_API_URL, params=params)
res = req.json()
[item['long_name'] for item in res['results'][0]['address_components'] if item['types'] == ['country', 'political']]

['Taiwan']

In [12]:
[item['long_name'] for item in res['results'][0]['address_components'] if item['types'] == ['country', 'political']]

['Taiwan']

In [0]:
{'results': [{'address_components': [{'long_name': 'Canterbury',
     'short_name': 'Canterbury',
     'types': ['locality', 'political']},
    {'long_name': 'Canterbury',
     'short_name': 'Canterbury',
     'types': ['postal_town']},
    {'long_name': 'Kent',
     'short_name': 'Kent',
     'types': ['administrative_area_level_2', 'political']},
    {'long_name': 'England',
     'short_name': 'England',
     'types': ['administrative_area_level_1', 'political']},
    {'long_name': 'United Kingdom',
     'short_name': 'GB',
     'types': ['country', 'political']}],
   'formatted_address': 'Canterbury, UK',
   'geometry': {'bounds': {'northeast': {'lat': 51.3061023, 'lng': 1.1237432},
     'southwest': {'lat': 51.25098550000001, 'lng': 1.0439585}},
    'location': {'lat': 51.280233, 'lng': 1.0789089},
    'location_type': 'APPROXIMATE',
    'viewport': {'northeast': {'lat': 51.3061023, 'lng': 1.1237432},
     'southwest': {'lat': 51.25098550000001, 'lng': 1.0439585}}},
   'place_id': 'ChIJiSyDDcVK2UcRivmDN1Pl8Z4',
   'types': ['locality', 'political']}],
 'status': 'OK'}

In [0]:
geo_location = {}

for loc in locations:
    params = {
    'address': loc,
    'key' : api_key    
    }
    req = requests.get(GOOGLE_MAPS_API_URL, params=params)
    res = req.json()
    
    if res['status'] == 'ZERO_RESULTS':
        geodata = []
        # put unknown into atlantic ocean
        geodata.append(14.5994)
        geodata.append(28.6731)
        geodata.append('location unknown')
        geodata.append('unknown')
 
    else:    
        result = res['results'][0]
        geodata = []
        geodata.append(result['geometry']['location']['lat'])
        geodata.append(result['geometry']['location']['lng'])
        geodata.append(result['formatted_address'])
        geodata.append(result['place_id'])
        geodata.append([item['long_name'] for item in res['results'][0]['address_components'] if item['types'] == ['country', 'political']][0] )

    
    geo_location[loc] = geodata
    

In [23]:
geo_location

{'Australia': [-25.274398,
  133.775136,
  'Australia',
  'ChIJ38WHZwf9KysRUhNblaFnglM',
  'Australia'],
 'Barcelona, Spain ': [41.3850639,
  2.1734035,
  'Barcelona, Spain',
  'ChIJ5TCOcRaYpBIRCmZHTz37sEQ',
  'Spain'],
 'Cairo, Egypt': [30.0444196,
  31.2357116,
  'Cairo, Cairo Governorate, Egypt',
  'ChIJ674hC6Y_WBQRujtC6Jay33k',
  'Egypt'],
 'Canterbury, United Kingdom ': [51.280233,
  1.0789089,
  'Canterbury, UK',
  'ChIJiSyDDcVK2UcRivmDN1Pl8Z4',
  'United Kingdom'],
 'Changhua Christian Hospital, Changhua City, Taiwan ': [24.0784606,
  120.5397995,
  'No. 176, Zhonghua Road, Changhua City, Changhua County, Taiwan 500',
  'ChIJa8aSeJA4aTQRiGd4FEqquCc',
  'Taiwan'],
 'China': [35.86166,
  104.195397,
  'China',
  'ChIJwULG5WSOUDERbzafNHyqHZU',
  'China'],
 'Cho Ray Hospital, Ho Chi Minh City, Vietnam': [10.7570661,
  106.6597319,
  '201B Nguyễn Chí Thanh, Phường 12, Quận 5, Hồ Chí Minh, Vietnam',
  'ChIJ1_fr7_EudTERWAqRuFPOFJA',
  'Vietnam'],
 'Civili Hospital, Vigevano, Italy': [4

In [18]:
with open('./datasets/geocoded.json', 'w') as fp:
    json.dump(geo_location, fp)

array(['Australia', 6], dtype=object)

In [0]:
with open('./geocoded.json', 'w') as fp:
  json.dump(geo_location, fp)

In [0]:
df_demog['country'] = df_demog['location'].apply(lambda x: geo_location[x][4] if x != '' else '')
df_demog['lonlat'] = df_demog['location'].apply(lambda x: [geo_location[x][0], geo_location[x][1]] if x!='' else [14.5994,28.6731])

In [77]:
df_demog[['patientid','modality','view']].groupby(['modality','view']).count() 	

patientid
modality view                    
CT       Axial                 41
         Coronal                3
X-ray    AP                    38
         AP Supine             29
         AP semi erect          1
         L                     27
         PA                   190

In [0]:
df_demog['conditions'] = df_demog['clinical_notes'].apply(lambda text: list(filter(lambda a: a != 'not',
                               ['diabetes' if any(i in text for i in ['DM', 'type 2 diabetes', 'T2DM', 'Type 2 diabetes', 'Type 1 diabetes', 'type 1 diabetes', 'T1DM']) else 'not',
                                'hiv' if any(i in text for i in ['HIV', 'HIV/AIDS', 'AIDS']) else 'not' ,
                                'std' if any(i in text.lower() for i in ['syphilis', ' gonorrhea', 'trichomoniasis', 'chlamydia'])  else 'not',
                                'alcoholism' if any(i in text.lower() for i in ['ETOH excess', 'alcoholism', 'alcohol misuse']) else 'not',
                                'chronic obstructive pulmonary disease' if any(i in text.lower() for i in ['copd', 'chronic obstructive pulmonary disease', 'bronchitis'])  else 'not' ,
                                'hemorrhoid' if any(i in text for i in ['IRC', 'Infrared Coagulation','hemorrhoid','Hemorrhoid', 'Infrared photocoagulation', 'rubber band ligation', 'coagulation therapy'])  else 'not' ,
                                'hypertension' if any(i in text.lower() for i in ['hypertension', 'hypertensive', 'blood pressure', 'high blood pressure', 'HTN'])  else 'not' ,
                                'atrial fibrillation' if any(i in text for i in ['AF', 'atrial flutter', 'atrial fibrillation','Atrial flutter', 'Atrial fibrillation'])  else 'not',
                                'asthma' if any(i in text.lower() for i in ['asthma'])  else 'not' ,
                                'renal failure' if any(i in text.lower() for i in ['dialysis', 'chronic end-stage renal failure', 'renal failure', 'renal', 'dysfunctional a-v fistula', 'vascath', 'arteriovenous fistula dysfunction', 'arteriovenous fistula', 'avf', 'av fistula'])  else 'not',
                                'immunosuppressed' if any(i in text.lower() for i in ['immunosuppressed', 'loss of viral suppression'])  else 'not',
                                'heart valve' if any(i in text.lower() for i in ['valvuloplasty', 'valvulotomy', 'valvotomy', 'narrowed heart valve', 'aortic stenosis', 'mitral valve replacement','mitral valve regurgitation', 'mitral valve stenosis', 'mitral stenosis', 'mitral degenerative disease'])  else 'not' ,
                                'pancreas' if any(i in text.lower() for i in ['pancreatectomy','pancreatitis','pancreas'])  else 'not',
                                'smoker' if any(i in text.lower() for i in ['tobacco', 'ex-smoker', 'smoker','smokes'])  else 'not',
                                'arteritis' if any(i in text.lower() for i in ["horton's arteritis", 'giant-cell arteritis' , 'gca', 'temporal arteritis', 'cranial arteritis'])  else 'not',
                                'pacemaker' if any(i in text.lower() for i in ['single-lead pacemaker', 'pacemaker' ])  else 'not',
                                'arrhythmias' if any(i in text.lower() for i in ["arrhythmia", ])  else 'not',
                                'coronary heart disease' if any(i in text.lower() for i in ['coronary bypass', 'heart bypass', 'bypass graft','CABG', 'coronary heart disease' ])  else 'not',
                                'chemotherapy' if any(i in text.lower() for i in ['chemotherapy'])  else 'not',
                                'cancer' if any(i in text.lower() for i in ['malignancy', 'cancer'])  else 'not',
                                'rheumatic' if any(i in text.lower() for i in ['rheumatic', 'rheumatic polymyalgia','rheumatoid arthritis', 'rheumatic disease', 'swollen joints', 'rheumatoid factor', 'pmr treatment'])  else 'not',
                                'cerebral palsy' if any(i in text.lower() for i in ['cerebral palsy'])  else 'not',
                                'acid reflux' if any(i in text.lower() for i in ['gastroesophageal reflux disease', 'gerd', 'acid reflux'])  else 'not'
                                ]
                                )))

In [0]:
df_demog['symptoms'] = df_demog['clinical_notes'].apply(lambda text: list(filter(lambda a: a != 'not',
                              ['fever' if any(i in text.lower() for i in ['fever', 'febrile', 'pyrexia', 'temperature'])  else 'not',
                              'mucus cough' if any(i in text.lower() for i in ['productive cough', 'phlegm', 'mucus', 'catarrh', 'wet cough'])  else 'not',
                              'dry cough' if any(i in text.lower() for i in ['dry cough',  'sore throat','non-productive cough'])  else 'not',
                              'cough' if any(i in text.lower() for i in ['cough'])  else 'not',
                              'breathing difficulties' if any(i in text.lower() for i in ['wheezing','breathing difficulties', 'breathing difficulty', 'dyspnea', 'respiratory effort', 'shortness of breath', 'disponoica', 'sob', 'labored breathing'])  else 'not',
                              'normal breathing' if any(i in text.lower() for i in ['eupnoic', 'eupnea', 'eupnoea','normal breathing','quiet breathing', 'resting respiratory rate' 'good breathing', 'unlabored breathing'])  else 'not',
                              'rapid breathing' if any(i in text.lower() for i in ['rapid breathing','tachypnea', 'tachypnoic', 'tachypnoea', 'tachypneic', 'tachipnoica'])  else 'not',
                              'flesh-eating disease' if any(i in text.lower() for i in ['necrotizing fasciitis', 'flesh-eating disease', 'flesh-eating bacteria', 'inflammatory infection of the fascia'])  else 'not',
                              'chest pain' if any(i in text.lower() for i in ['chest pain'])  else 'not',
                              'muscle pain' if any(i in text.lower() for i in ['myalgia', 'myalgias','muscle ache', 'muscle pain'])  else 'not',
                              'abdominal pain' if any(i in text.lower() for i in ['abdominal pain', 'stomach ache'])  else 'not',
                              'painful swallowing' if any(i in text.lower() for i in ['odynophagia', 'painful swallowing'])  else 'not',
                              'diarrhea' if any(i in text.lower() for i in ['diarrhea']) else 'not' ,
                              'heart failure' if any(i in text.lower() for i in ['heart failure', 'congestive cardiac failure', 'ccf', 'heart is enlarged', 'enlarged heart'])  else 'not' ,
                              'rapid heart beat' if any(i in text.lower() for i in ['tachycardia','supraventricular', 'svt', 'tachyarrhythmia'])  else 'not',
                              'confusion' if any(i in text.lower() for i in ['confusion'])  else 'not',
                              'fatigue' if any(i in text.lower() for i in ['fatigue', 'lethargy', 'malaise', 'asthenia', 'tired', 'tiredness'])  else 'not',
                              'nausea' if any(i in text.lower() for i in ['nausea', 'vomit', 'dizziness', 'queasy'])   else 'not',
                              'sepsis' if any(i in text.lower() for i in ['sepsis', 'sepsic shock'])  else 'not',
                              'weight loss' if any(i in text.lower() for i in ['weight loss'])  else 'not',
                              'flu-like' if any(i in text.lower() for i in ['flu-like illness', 'flu like symptoms'])  else 'not',
                              'common-cold-like' if any(i in text.lower() for i in ['coryzal symptoms', 'coryza', 'common cold', 'inflammation of the mucosa'])  else 'not' ,
                              'hypotension' if any(i in text.lower() for i in ['hypotension'])  else 'not',
                              'rash' if any(i in text.lower() for i in ['rash'])  else 'not',
                              'depression' if any(i in text.lower() for i in ['dysthymic', 'dysthymia', 'depression'])  else 'not',
                              'rash' if any(i in text.lower() for i in ['rash'])  else 'not',
                              'respiratory failure' if any(i in text.lower() for i in ['moderate respiratory failure', 'acute respiratory failure', 'respiratory failure'])  else 'not'
                 ])))

In [0]:
df_demog['image_url'] = df_demog['filename'].apply(lambda x: 'https://raw.github.com/ieee8023/covid-chestxray-dataset/master/images/'+x)

In [41]:
df_demog[df_demog['conditions'].apply(len) != 0][['image_url','date','offset', 'sex', 'age', 'finding', 'survival', 'intubated', 'temperature','pO2_saturation','clinical_notes','conditions','symptoms']].head(200)

,image_url,date,offset,sex,age,finding,survival,intubated,temperature,pO2_saturation,clinical_notes,conditions,symptoms
59,https://raw.github.com/ieee8023/covid-chestxra...,"May 10, 2015",0,M,65,ARDS,Unknown,Unknown,36.5,0.0,"Admitted to ICU with necrotizing fasciitis, se...",[renal failure],"[flesh-eating disease, respiratory failure]"
86,https://raw.github.com/ieee8023/covid-chestxra...,2019,0,F,61,No Finding,Y,N,37.8,98.0,"Female, 61 years old, smoker. In November 2019...",[smoker],"[fever, cough, fatigue]"
87,https://raw.github.com/ieee8023/covid-chestxra...,2019,0,F,61,No Finding,Y,N,37.8,98.0,"Female, 61 years old, smoker. In November 2019...",[smoker],"[fever, cough, fatigue]"
95,https://raw.github.com/ieee8023/covid-chestxra...,2020,0,F,71,COVID-19,Unknown,Unknown,36.5,97.0,"Woman, 71 years old, reports dyspnea and fever...","[diabetes, chronic obstructive pulmonary disea...","[fever, breathing difficulties, rapid breathing]"
96,https://raw.github.com/ieee8023/covid-chestxra...,2020,5,F,55,COVID-19,Unknown,Unknown,36.5,70.0,"Woman, 55 years old, reports dyspnea for a few...",[asthma],"[fever, breathing difficulties]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
326,https://raw.github.com/ieee8023/covid-chestxra...,NaN,2,F,30,COVID-19,Unknown,Y,36.5,0.0,A 30-year-old female patient with a past medic...,"[hypertension, asthma]","[fever, cough, breathing difficulties]"
331,https://raw.github.com/ieee8023/covid-chestxra...,NaN,1,M,73,COVID-19,Y,Y,36.5,0.0,History 73 year old male with aorta insufficie...,[pacemaker],"[fever, cough]"
332,https://raw.github.com/ieee8023/covid-chestxra...,NaN,4,M,73,COVID-19,Y,Y,36.5,0.0,History 73 year old male with aorta insufficie...,[pacemaker],"[fever, cough]"
333,https://raw.github.com/ieee8023/covid-chestxra...,NaN,8,M,73,COVID-19,Y,Y,36.5,0.0,History 73 year old male with aorta insufficie...,[pacemaker],"[fever, cough]"


In [0]:
df_demog['date_parsed'] = df_demog['date'].apply(lambda x: dateparser.parse(str(x)))

In [43]:
df_demog[['date','date_parsed']]

,date,date_parsed
0,"January 22, 2020",2020-01-22
1,"January 25, 2020",2020-01-25
2,"January 27, 2020",2020-01-27
3,"January 28, 2020",2020-01-28
4,"January 25, 2020",2020-01-25
...,...,...
333,NaN,2020-08-18
334,NaN,2020-08-18
335,2020,2020-04-18
336,2020,2020-04-18


In [44]:
def totimestamp(dt, epoch=datetime(1970,1,1)):
    td = dt - epoch
    return int(1000*(td.microseconds + (td.seconds + td.days * 86400) * 10**6) / 10**6) 

now = datetime.utcnow()
print (now)
print (totimestamp(now))

2020-04-18 13:21:46.599017
1587216106599


In [0]:
df_demog['unix_timestamp'] = df_demog['date_parsed'].apply(lambda x : totimestamp(x))

In [0]:
df_demog['idx'] =  pd.Series(np.arange(len(df_demog))).values

In [0]:
df_es = df_demog[['idx','unix_timestamp', 'image_url','offset', 'sex', 'age', 'country', 'finding', 'survival', 'intubated', 'temperature', 'pO2_saturation','clinical_notes','conditions','symptoms', 'date_parsed']]

In [48]:
df_es.groupby(['survival']).count()

,idx,unix_timestamp,image_url,offset,sex,age,country,finding,intubated,temperature,pO2_saturation,clinical_notes,conditions,symptoms,date_parsed
survival,,,,,,,,,,,,,,,
N,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26
Unknown,230,230,230,230,230,230,230,230,230,230,230,230,230,230,230
Y,82,82,82,82,82,82,82,82,82,82,82,82,82,82,82


# ES Load

In [67]:
# url and credentials go into service account's .bashrc or profile in it's home directory
os.environ['BONSAI_URL'] = bonsai_url #
port = 443 #config.bonsai_port
bonsai = os.environ['BONSAI_URL']

auth = re.search('https\:\/\/(.*)\@', bonsai).group(1).split(':')

host = bonsai.replace('https://{}:{}@'.format(auth[0], auth[1]), '')


es_header = [{
    'host': host,
    'port' : port,
    'use_ssl': True,
    'http_auth': (auth[0], auth[1])
}]

es = Elasticsearch(es_header)

es.ping()

True

In [68]:
# check version compatibility with python's elasticsearch library
es.info()

{'cluster_name': 'elasticsearch',
 'cluster_uuid': 'zPvVm7AwTr6s-LfIjiRyLA',
 'name': 'ip-172-31-77-150',
 'tagline': 'You Know, for Search',
 'version': {'build_date': '2019-06-20T15:54:18.811730Z',
  'build_flavor': 'oss',
  'build_hash': '508c38a',
  'build_snapshot': False,
  'build_type': 'tar',
  'lucene_version': '8.0.0',
  'minimum_index_compatibility_version': '6.0.0-beta1',
  'minimum_wire_compatibility_version': '6.8.0',
  'number': '7.2.0'}}

In [98]:
es.indices.get_alias("*")

{}

In [104]:
es.indices.delete(index='virus', ignore=[400, 404])
#es.indices.delete(index='.kibana_1', ignore=[400, 404])

{'acknowledged': True}

In [70]:
df_es.columns

Index(['idx', 'unix_timestamp', 'offset', 'sex', 'age', 'finding', 'survival',
       'intubated', 'temperature', 'pO2_saturation', 'clinical_notes',
       'conditions', 'symptoms'],
      dtype='object')

In [116]:
# "Location_Coordinates": {"type": "geo_point",
# "ignore_malformed": "true"},
# "Event_Location" : {"type": "keyword"},
                # "location_coordinates": {"type": "geo_point",
                #                          "ignore_malformed": "true"},

request_body =  {
    "settings" : {
        "number_of_shards": 1,
        "number_of_replicas": 1
    },
    "mappings": {
            "properties": {
                "unix_timestamp": {"type": "date"},
                "image_url": {"type": "text"},
                "offset": {"type": "byte"},
                "sex": {"type": "keyword"},
                "age": {"type": "byte"},
                "country": {"type": "keyword"},
                "finding": {"type": "keyword"},
                "survival": {"type": "keyword"},
                "intubated": {"type": "keyword"},
                "temperature": {"type": "half_float"},
                "pO2_saturation": {"type": "half_float"},
                "clinical_notes": {"type": "text", 
                             "index_phrases" : "true"},  
                "conditions": {"type": "keyword"},
                "symptoms" : {"type": "keyword"},
                "date_parsed" : {"type": "date"}
            }
    } 
}


es.indices.create(index = 'virus1', body = request_body)

{'acknowledged': True, 'index': 'virus1', 'shards_acknowledged': True}

In [117]:
es.indices.get_mapping(index='virus1')

{'virus1': {'mappings': {'properties': {'age': {'type': 'byte'},
    'clinical_notes': {'index_phrases': True, 'type': 'text'},
    'conditions': {'type': 'keyword'},
    'date_parsed': {'type': 'date'},
    'finding': {'type': 'keyword'},
    'image_url': {'type': 'text'},
    'intubated': {'type': 'keyword'},
    'offset': {'type': 'byte'},
    'pO2_saturation': {'type': 'half_float'},
    'sex': {'type': 'keyword'},
    'survival': {'type': 'keyword'},
    'symptoms': {'type': 'keyword'},
    'temperature': {'type': 'half_float'},
    'unix_timestamp': {'type': 'date'}}}}}

In [0]:
bulk_data = []

for index, row in df_es.iterrows():
    data_dict = {}
    
    for i in range(len(row)):
        
        data_dict[df_es.columns[i]] = row[i]        
 
        doc_dict = {
                "_index": 'virus1',
                "_id": data_dict['idx'],
                "_source": data_dict
        }
    
    bulk_data.append(doc_dict)

In [0]:
res = helpers.bulk(client = es, actions = bulk_data)

In [120]:
es.search(body={"query": {"match_all": {}}}, index = 'virus1')["hits"]["hits"]

[{'_id': '0',
  '_index': 'virus1',
  '_score': 1.0,
  '_source': {'age': 65,
   'clinical_notes': 'infiltrate in the upper lobe of the left lung',
   'conditions': [],
   'date_parsed': '2020-01-22T00:00:00',
   'finding': 'COVID-19',
   'idx': 0,
   'image_url': 'https://raw.github.com/ieee8023/covid-chestxray-dataset/master/images/auntminnie-a-2020_01_28_23_51_6665_2020_01_28_Vietnam_coronavirus.jpeg',
   'intubated': 'Unknown',
   'offset': 0,
   'pO2_saturation': 0.0,
   'sex': 'M',
   'survival': 'Y',
   'symptoms': [],
   'temperature': 36.5,
   'unix_timestamp': 1579651200000},
  '_type': '_doc'},
 {'_id': '1',
  '_index': 'virus1',
  '_score': 1.0,
  '_source': {'age': 65,
   'clinical_notes': 'progressive infiltrate and consolidation',
   'conditions': [],
   'date_parsed': '2020-01-25T00:00:00',
   'finding': 'COVID-19',
   'idx': 1,
   'image_url': 'https://raw.github.com/ieee8023/covid-chestxray-dataset/master/images/auntminnie-b-2020_01_28_23_51_6665_2020_01_28_Vietnam_co

In [0]:
## Radiology terms:
# 'infiltrate' ('progressive infiltrate', 'diffuse infiltrates'
# 'consolidation' ('consolidation has expanded and become more dense', 'extensive consolidation', 'central airspace consolidation and residual peripheral consolidation','Vertical air space consolidation' )
# 'consolidative changes'
# 'opacities', 'opacity', 'opacifications', 'opacification' ('progressive diffuse interstitial opacities', 'ground-glass opacity', 'bilateral multifocal opacities', 'basilar opacity', 'streaky opacities', 'infrahilar airspace opacities', 'ill-defined patchy opacities')
# 'ground glass' ('diffuse ground glass opacities')
# 'ARDS'
# 'shadows' ('multiple patchy shadows', 'condensation shadows')
# 'brightness' ('brightness of both lungs was decreased')
# 'blurred' ('edges were blurred')
# 'peribronchovascular'
# 'thoracic abnormalities'
# 'acute intrathoracic plain-film abnormality'
# 'perihilar infiltration'
# 'GGOs ('persistent multifocal GGOs')', 'predominant perihilar mixed GGO'
# 'superimposed reticulation' 
# 'fibrotic change', 'mild fibrotic change', 'fibrosis'
# 'interlobular septal thickening'
# 'Paraseptal emphysema'
# 'bibasilar sparing'
# 'Incidental pleural nodular calcific plaques'
# 'alveolar consolidations'
# 'bulging of the horizontal fissure'
# 'fine reticular interstitial pulmonary pattern'
# 'septal thickening'
# 'reticular interstitial markings'
# 'cyst'
# 'symmetric interstitial patten'
# 'hyperlucent patches'
# 'pulmonary nodules'
# 'interlobular septal thickening'
# '"crazy paving" pattern'
# 'cystic spaces'
# 'Right internal jugular central venous line'
# 'pleural effusion'
# 'bilateral parenchymal thickenings'
# 'peribroncovascular interstitial texture'
# 'obliteration of the costophrenic sinus'
# 'reticulation'
# 'reversed halo sign'
# 'crazy paving appearance'
# 'inter/intralobular septal thickening'
# 'Ground glass nodule'
# 'air-space shadowing' 
# 'air bronchogram' 
# 'pneumomediastinum'
# 'pulmonary lesions'
# 'lung markings'
# 'prominent bilateral perihilar infiltration'



## Diagnosis indicators
# 'pneumonia'
# 'COVID-19 pneumonia'
# 'SARS-CoV-2 RNA'
# 'pneumocystis jiroveci pneumonia'
# 'pneumatocele'
# "P'neumocystis jiroveci"
# 'pneumocystis carinii pneumonia (PCP)'
# 'Legionella pneumophila'
# 'Legionella-Ag'
# 'SARS-CoV-2'
# 'organizing pneumonia'
# 'right upper lobe pneumonia'
# 'COVID-2019'
# 'SARS, type 1 pattern'
# 'SARS-CoV2 RNA'
# 'COVID-19 pneumonia'




## Negations
# 'No significant findings reported'
# 'No evidence of mediastinal adenopathy'
# 'No pleural abnormality'
# 'no fever or chest pain'
# 'No mediastinal lymphadenopathy has been seen'
# 'No significant mediastinal lymphadenopathy'
# 'No pleural effusion or adenopathy'
# 'No respiratory symptoms were reported'
# 'These findings are nonspecific'
# 'No pleural effusion'
# 'No pleural abnormality'
# 'No pneumothorax is evident'
# 'there is no collapse and no pleural effusion'
# 'No nodules, nodes or pleural effusions were seen. The heart was not enlarged'
# 'without need for oxygen supplementation'
# 'No significant findings'
# 'no coronavirus test was performed because he did not meet epidemiological criteria'
# 'No collapse or consolidation. No pleural effusion.
# 'No intubation'
# 'not on any regular medications'

## Pre-existing conditions and treatments (anamnesis, PMH) 
# 'COPD'
# 'IRC'
# 'arterial hypertension', 'essential hypertension'
# 'DM', 'type 2 diabetes', 'T2DM'
# 'mitral valve replacement'
# 'Single-lead pacemaker' 
# 'valvuloplasty'
# 'Previous coronary bypass surgery'
# 'rheumatic polymyalgia'
# "Horton's arteritis"
# 'ex-smoker'
# 'HIV', 'HIV/AIDS'
# 'immunosuppressed', 'loss of viral suppression'
# 'Syphilis'
# 'chemotherapy'
# 'hematological malignancy', 'prostate cancer'
# 'asthma'
# 'cerebral palsy'
# 'ETOH excess', 'alcoholism', 'alcohol misuse'
# 'pancreatectomy'
# 'GERD'
# 'dialysis', 'chronic end-stage renal failure', 'renal failure', 'dysfunctional A-V fistula', 'vascath'
# 'upper abdominal surgical clips'
# 'AF', 'atrial flutter', 'atrial fibrillation'
# 

## Measurements and tests:
# 'elevated c-reactive protein', 'elevated CRP', 'high CRP', 'CRP 190 mg/L'
# '97% pO2', 'saturation of 70%', 'pO2 93%', 'low oxygen saturation' , 'SpO2'
# 'PAFI'
# 'bronchoscopy'
# 'PCR' polymerise chain reaction
# 'CD4 count less than 200 cells/mm3' , 'CD4 count of 22 cells/mm3, 'low CD4 count'
# 'Febrile neutropenia'
# 'lymphopenia', 'lymphocytopenia'
# 'crackles', 'crepitations'
# 'hypoxia'




## Intubation - an indicator of severety if not labelled
# Ventilation
# ET tube insertion in ventilated victim
# Remains intubated
# ET tube, right jugular central line and NG tube in situ
# New oxygen requirement
# ETT and NG tube placed
# ETT and NG tube in place
# 'requiring intensive care admission for ventilation and inotropic support' 
# 'Support lines (ETT, NG, and left internal jugular CVC) are in situ'
# Appropriately positioned right central venous catheter, endotracheal and nasogastric tubes
# Endotracheal tube and central venous line 


# Symptoms:
# 'fever', 'febrile', 'pyrexia', 'temperature'
# 'apyretic'
# 'productive cough'
# 'sore throat'
# 'cough'
# 'dry cough'
# 'non-productive cough'
# 'breathing difficulties', 'breathing difficulty', 'dyspnea', 'respiratory effort', 'shortness of breath', 'disponoica', 'SOB', 'LABORED BREATHING'
# 'wheezing'
# 'eupnoic', 'eupnea', 'eupnoea'
# 'tachypnea', 'tachypnoic', 'tachypnoea', 'tachypneic', 'tachipnoica'
# 'necrotizing fasciitis'
# 'chest pain'
# 'myalgia', 'myalgias'
# 'abdominal pain'
# 'odynophagia'
# 'diarrhea'
# 'heart failure'
# 'heart is enlarged'
# 'tachycardia'
# 'confusion'
# 'fatigue', 'lethargy', 'malaise', 'asthenia'
# 'nausea'
# 'sepsis'
# 'confusion'
# 'weight loss'
# 'flu-like illness', 'flu like symptoms'
# 'rash'
# 'yellow mucus'
# 'stable'
# 'worsening', 'clinical worsening'
# 'dysthymic', 'dysthymia'
# 'moderate respiratory failure', 'acute respiratory failure'
# 'hypoxia'
# 'hypotension'
# 'coryzal symptoms', 'coryza', 'common cold', 'inflammation of the mucosa'

In [0]:
#processed = pipeline.fit(df_demog)